# 6.1 Systematic Model Comparison Framework - Code Brief

Condensed reference for comparing all model families.

## Setup

In [ ]:
import numpy as np
import pandas as pd
import time
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier
from sklearn.neural_network import MLPClassifier
from sklearn.metrics import (
    accuracy_score, precision_score, recall_score, f1_score,
    roc_auc_score, roc_curve, precision_recall_curve, average_precision_score,
    brier_score_loss, log_loss
)
from xgboost import XGBClassifier
from lightgbm import LGBMClassifier

## Model Definitions

In [ ]:
# Dictionary to store models and results
models = {}
training_times = {}

# Logistic Regression (L2)
models['Logistic Regression (L2)'] = LogisticRegression(
    penalty='l2', C=0.1, solver='lbfgs', max_iter=1000, random_state=42
)

# Decision Tree
models['Decision Tree'] = DecisionTreeClassifier(
    max_depth=8, min_samples_split=20, min_samples_leaf=10,
    class_weight='balanced', random_state=42
)

# Random Forest
models['Random Forest'] = RandomForestClassifier(
    n_estimators=200, max_depth=12, min_samples_split=10,
    class_weight='balanced', n_jobs=-1, random_state=42
)

# Gradient Boosting
models['Gradient Boosting'] = GradientBoostingClassifier(
    n_estimators=150, learning_rate=0.1, max_depth=5, random_state=42
)

# XGBoost
models['XGBoost'] = XGBClassifier(
    n_estimators=150, learning_rate=0.1, max_depth=5,
    eval_metric='logloss', random_state=42
)

# LightGBM
models['LightGBM'] = LGBMClassifier(
    n_estimators=150, learning_rate=0.1, max_depth=5,
    class_weight='balanced', random_state=42, verbose=-1
)

# Neural Network
models['Neural Network'] = MLPClassifier(
    hidden_layer_sizes=(64, 32, 16), activation='relu', solver='adam',
    alpha=0.001, max_iter=500, early_stopping=True, random_state=42
)

## Train All Models

In [ ]:
# Models that require scaled features
scaling_required = {
    'Logistic Regression (L2)': True,
    'Decision Tree': False,
    'Random Forest': False,
    'Gradient Boosting': False,
    'XGBoost': False,
    'LightGBM': False,
    'Neural Network': True
}

# Train each model
for model_name, model in models.items():
    start_time = time.time()
    if scaling_required.get(model_name, False):
        model.fit(X_train_scaled, y_train)
    else:
        model.fit(X_train, y_train)
    training_times[model_name] = time.time() - start_time

## Evaluate All Models

In [ ]:
def evaluate_model(model, X_test, y_test, model_name, requires_scaling=False, X_test_scaled=None):
    X_eval = X_test_scaled if requires_scaling else X_test
    
    y_pred = model.predict(X_eval)
    y_prob = model.predict_proba(X_eval)[:, 1]
    
    return {
        'Model': model_name,
        'Accuracy': accuracy_score(y_test, y_pred),
        'Precision': precision_score(y_test, y_pred),
        'Recall': recall_score(y_test, y_pred),
        'F1 Score': f1_score(y_test, y_pred),
        'ROC-AUC': roc_auc_score(y_test, y_prob),
        'Avg Precision': average_precision_score(y_test, y_prob)
    }, y_pred, y_prob

# Evaluate all models
all_results = []
probabilities = {}

for model_name, model in models.items():
    requires_scaling = scaling_required.get(model_name, False)
    metrics, y_pred, y_prob = evaluate_model(
        model, X_test, y_test, model_name,
        requires_scaling=requires_scaling,
        X_test_scaled=X_test_scaled
    )
    all_results.append(metrics)
    probabilities[model_name] = y_prob

results_df = pd.DataFrame(all_results).set_index('Model')
results_df['Training Time (s)'] = results_df.index.map(training_times)

## ROC Curve Comparison

In [ ]:
# Get ROC curves for all models
roc_curves = {}
for model_name, y_prob in probabilities.items():
    fpr, tpr, _ = roc_curve(y_test, y_prob)
    auc = roc_auc_score(y_test, y_prob)
    roc_curves[model_name] = {'fpr': fpr, 'tpr': tpr, 'auc': auc}

## Model Selection Summary

In [ ]:
# Rank models by AUC
ranked_df = results_df.sort_values('ROC-AUC', ascending=False)

# Best model by different criteria
print(f"Best ROC-AUC: {results_df['ROC-AUC'].idxmax()}")
print(f"Best F1 Score: {results_df['F1 Score'].idxmax()}")
print(f"Best Recall: {results_df['Recall'].idxmax()}")
print(f"Fastest Training: {min(training_times, key=training_times.get)}")

## Key Concepts

| Model | Interpretability | Best For |
|:------|:-----------------|:---------|
| Logistic Regression | High | Compliance, reports |
| Decision Tree | Very High | Advisor tools |
| Random Forest | Medium | Balanced needs |
| Gradient Boosting | Low | Max performance |
| Neural Network | Low | Complex patterns |